In [1]:
from sympy import *
from aux_methods import show_expr, evolv

In [2]:
# Define symbols to use
k, N, u, r, g = symbols('k, N, mu, r, gamma')

# This list of symbols represent values for the burst distribution
B = [symbols('B_{%d}' % i) for i in range(20)]

steps = 2
burst_size = bs = 1

# the mRNA transition types
# list key corresponds to mRNA state
tran_types = [steps*u] * steps # we take same rate between consecutive degradation steps
show_expr(tran_types)

<IPython.core.display.Math object>

In [3]:
# protein production at each mRNA state
prod_types = [r] * steps
# production rate at each mRNA degradation step for a single mRNA
# the total rate will depend of the number of mRNAs present in a state
show_expr(prod_types)

<IPython.core.display.Math object>

In [4]:
# initial transition always from ground state 
# this is a tuple of size equal to the number of possible mRNA states
# ground = (0,) for the simplest case
# ground = (0, 0) if there are two possible mRNA states
ground = (0, ) * len(tran_types)

# an arrival from ground state to first mRNA state
first = (bs, ) + (0, ) * (len(tran_types) - 1)

# transition rates between mRNA states
tran_rates = {
    (ground, first): k/N,
    #(ground, first): B[bs]*k/N # include a rescaling in arrival rates
}

In [5]:
# will contain the actual protein production rates for all posible
# states of the system
prod_rates = {}

# the set of posible states of the system
states = set()

# auxiliar set, used to build the final states set
n_states = set()
n_states.add(first)

# posible states are discovered by exploring how the arrival
# state evolves, according to function evolv(state)
while len(n_states) > 0:
    t_state = n_states.pop()
    # make this state part of the set
    states.add(t_state)
    try:
        # discover child states for this one
        childs = evolv(t_state)
        
        for child, i in childs:
            # transition rate is proportional to the
            # number of mRNAs in ith state
            tran_rates[(t_state, child)] = t_state[i]*tran_types[i]
            n_states.add(child)
    except IndexError:
        pass

# protein production rates at each state
for state in states:
    # initialize
    prod_rates[state] = 0
    for i, n in enumerate(state):
        # add up all protein production rate contributions
        # n is the number of mRNAs in i-th mRNA state
        prod_rates[state] += n * prod_types[i]

In [6]:
# Determine the set of unique nodes
nodes = set([node for edge in tran_rates.keys() for node in edge])

# Node count, sorted list of nodes
n_cnt = len(nodes)
n_lst = sorted(list(nodes))
n_idx = range(n_cnt)

# dict map from node name to node index in sorted list
n_dct = dict(zip(n_lst, n_idx))

In [7]:
# Build the "transition" matrix
K = zeros(n_cnt, n_cnt)
for t, r in tran_rates.items():
    K[n_dct[t[1]], n_dct[t[0]]] += r
    K[n_dct[t[0]], n_dct[t[0]]] -= r
show_expr(K)

<IPython.core.display.Math object>

In [8]:
R = zeros(n_cnt, n_cnt)
for s, r in prod_rates.items():
    R[n_dct[s], n_dct[s]] += r
show_expr(R)

<IPython.core.display.Math object>

In [9]:
X = K.copy()
X.row_del(0)
X = X.row_insert(0, ones(1, K.shape[0]))
G = eye(K.shape[0])*g
kr = R*ones(K.shape[0], 1)
b = zeros(K.shape[0], 1)
b[0] = 1

In [10]:
m0 = factor(X.LUsolve(b))
m1 = factor((K-G).LUsolve(-R*m0))

In [11]:
# 1st moment at reduced model
mean_rm = factor((kr.T * m0/g)[0])
show_expr(mean_rm)

<IPython.core.display.Math object>

In [12]:
# 1st moment: E[p], the mean
# for protein number at original model
mean = (N*mean_rm).limit(N, S.Infinity)
show_expr(mean)

<IPython.core.display.Math object>

In [13]:
# 2nd moment at reduced model
secm_rm = factor((kr.T * m1/g)[0])
show_expr(secm_rm)

<IPython.core.display.Math object>

In [14]:
# 2nd moment: E[p*(p-1)]
# for protein number at original model
secm = factor(mean**2 + (N*secm_rm).limit(N, S.Infinity))
show_expr(secm)

<IPython.core.display.Math object>

In [15]:
# compute the fano factor
# for protein number at original model
variance = factor(secm + mean - mean**2)
fano = factor(variance/mean -1) +1 # tweak factorization
show_expr(fano)

<IPython.core.display.Math object>

In [16]:
# compute the fano factor
# for protein number at original model
variance = factor(secm + mean - mean**2)
fano = factor(variance/mean -1) +1 # tweak factorization
show_expr(fano)

<IPython.core.display.Math object>